## 1. Execute imports

In [166]:
import pandas as pd
import numpy as np
from patsy import ModelDesc, dmatrices, dmatrix, demo_data
import re
import pprint
import json

## 2. Create complex operations dict

In [167]:
# TODO: add more complex operations from numpy
COMPLEX_OPERATIONS = {
    'cos': 'np.cos',
    'tan': 'np.tan',
    'log': 'np.log',
    'log10': 'np.log10',
    'log2': 'np.log2',
}

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## 3. Execute functions

In [168]:
def add_blank_spaces_to_formula(formula: str) -> str:
    new = ''
    for index, element in enumerate(formula):
        next_idx = index + 1
        if next_idx < len(formula):
            if not re.match('\w', formula[index+1]):
                new += element + ' '
            else:
                new += element
        else:
            new += element + ' '
    return new

In [169]:
def clean_formula(formula: str) -> str:
    result = formula
    for operation in COMPLEX_OPERATIONS:
        if(operation in formula):
            result = formula.replace(operation, "")
    return result

def get_formula_variables(formula: str):
  '''
  Returns a list of every variable (non repeated) from the formula
  '''
  cleaned_formula = clean_formula(formula)
  return sorted(list(set("".join(re.findall("[a-zA-Z]+", cleaned_formula)))))

def group_columns(formula: str, data: pd.DataFrame):
  # get number of variables inside formula
  # convert string to set that only holds unique elements
  characters = get_formula_variables(formula=formula)
  
  # get dataset number of columns
  columns = len(data.columns)
  columns_lst = list(data.columns)
  characters_len = len(characters)

  result = []
  
  # column by column
  for i in range(0, columns):  
    # current column + 1 and substract 1 from characters so we don't count current character
    for j in range(i+1, columns, characters_len-1):
      column_variables = [columns_lst[i]]
      column_variables.extend(columns_lst[j:j+(characters_len-1)])
      # compare numbers and group columns by number of variables inside the formula
      if(len(column_variables) == characters_len):
        result.append(column_variables)
  return result # grouped columns

In [170]:
def get_formula_by_columns(formula: str, columns: list) -> dict:
  '''
  Mapping every single formula's variable to a column.
  '''
  to_replace = {}

  # formula variables
  variables = get_formula_variables(formula=formula)
  # iterate over grouped columns
  for cidx, column_group in enumerate(columns):
    formula_grouped = {}
    # iterate over variables
    for idx, variable in enumerate(variables):
      # variable paired to column name
      formula_grouped[variable] = column_group[idx]
    # every column group represents a key
    to_replace[cidx] = formula_grouped
  return to_replace

In [171]:
def parse_formula(formula: str, formula_columns: dict) -> list:
  '''
  Parses, effectively, every grouped column to a real formula. 
  In simple words, replaces every formula variable for its paired column.
  '''
  result = []
  formula_variables = re.findall(r'\w+', formula)

  for variables_paired in formula_columns.values():
        new_formula = formula
        for variable in formula_variables:
            if variable in variables_paired:
                # we need to put a blank space after a single character, 
                # so we can identify it then with the regex
                replace_regex = f'{variable}(?:[^\w\*\\\+\(\)])'
                new_formula = re.sub(replace_regex, variables_paired[variable], new_formula)
            elif variable in COMPLEX_OPERATIONS:
                new_formula = new_formula.replace(variable, COMPLEX_OPERATIONS[variable])
        new_formula = new_formula.replace(" ", "")

        result.append(new_formula)
  
  return result

In [172]:
def execute_formula(formula_by_columns: list, data: pd.DataFrame) -> pd.DataFrame:
  '''
  Take every real formula and executes it via patsy dmatrix.
  Saves every formula result inside a new dataframe's column.
  '''
  new_df = data.copy()
     
  for formula_columns in formula_by_columns:
    result_items = []
    add_data = True
    try:
        formula = "I("+formula_columns+")-1"
        result = dmatrix(formula, data, NA_action='raise')
        for item in result:
            result_items.append(item.item())
    except:
        # Ignore Patsy error.
        add_data = False
        
    if add_data:
        if "np." in formula_columns:
            new_df[formula_columns.replace('np.', '')] = result_items
        else:
            new_df[formula_columns] = result_items
    else:
        print(f"{bcolors.WARNING}Your data has some invalid values. Script will ignore them and their possible result.{bcolors.ENDC}")
        
  return new_df

In [177]:
def execute(formula_input: str, data: pd.DataFrame) -> pd.DataFrame:
    
    formula = add_blank_spaces_to_formula(formula_input.lower())
    grouped_columns = group_columns(formula, data)
    
    replaceable_result = get_formula_by_columns(formula, grouped_columns)

    executable_formulas = parse_formula(formula, replaceable_result)
    new_data = execute_formula(executable_formulas, data)
    
    return new_data

## 4. Play around
Just execute the function `execute` that receives a formula and a pandas' dataframe

In [178]:
df = pd.read_csv('winequality-white.csv', delimiter=';')
x = df.drop(['quality'], axis=1)
x.columns = x.columns.str.replace(' ','_')

execute(formula_input="(a*cos(b))/c", data=x).head(10)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,...,(residual_sugar*cos(pH))/sulphates,(chlorides*cos(free_sulfur_dioxide))/total_sulfur_dioxide,(chlorides*cos(density))/pH,(chlorides*cos(sulphates))/alcohol,(free_sulfur_dioxide*cos(total_sulfur_dioxide))/density,(free_sulfur_dioxide*cos(pH))/sulphates,(total_sulfur_dioxide*cos(density))/pH,(total_sulfur_dioxide*cos(sulphates))/alcohol,(density*cos(pH))/sulphates,(pH*cos(sulphates))/alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,...,-45.539655,0.000139,0.008092,0.004605,42.167596,-98.999250,30.569432,17.395001,-2.202183,0.306971
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,...,-3.224424,0.000051,0.008097,0.004551,14.064649,-28.213708,21.813655,12.259783,-2.003173,0.306495
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,...,-15.572015,0.000080,0.008350,0.004479,-27.891092,-67.704412,16.198971,8.689199,-2.245755,0.292029
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,...,-21.225108,-0.000309,0.009891,0.005396,-37.694323,-117.362360,31.719095,17.304782,-2.486084,0.296786
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,...,-21.225108,-0.000309,0.009891,0.005396,-37.694323,-117.362360,31.719095,17.304782,-2.486084,0.296786
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,...,-15.572015,0.000080,0.008350,0.004479,-27.891092,-67.704412,16.198971,8.689199,-2.245755,0.292029
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,...,-14.882633,0.000051,0.007706,0.004179,-18.470647,-63.782715,23.290503,12.630551,-2.115247,0.295332
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,...,-45.539655,0.000139,0.008092,0.004605,42.167596,-98.999250,30.569432,17.395001,-2.202183,0.306971
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,...,-3.224424,0.000051,0.008097,0.004551,14.064649,-28.213708,21.813655,12.259783,-2.003173,0.306495
9,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,...,-3.323092,-0.000328,0.007454,0.003602,-27.642337,-62.031058,21.854243,10.559789,-2.201659,0.263585


In [179]:
df2 = pd.DataFrame(np.array([[0, 2, 3], [-1, -1, 6], [7, 8, 9]]), columns=['c1', 'c2', 'c3'])
df2

,c1,c2,c3
0,0,2,3
1,-1,-1,6
2,7,8,9


In [181]:
execute(formula_input="LOG(b+a)", data=df2)

Your data has some invalid values. Script will ignore them and their possible result.


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,c1,c2,c3,log(c3+c1),log(c3+c2)
0,0,2,3,1.098612,1.609438
1,-1,-1,6,1.609438,1.609438
2,7,8,9,2.772589,2.833213


<ipython-input-24-03f1b4faceaf>:1: RuntimeWarning: invalid value encountered in log
  np.log(0+-1)


nan